In [3]:
import pandas as pd
import numpy as np

In [4]:
from backprop import *
from cost_functions import *
from evaluation_metrics import *
from utils import *

In [5]:
def prepare_df(data):

    remove_cols = ['Name', 'Ticket', 'Cabin']
    data = data.drop(remove_cols, axis=1)
    
    for col in ['Sex', 'Embarked']:
        data = data.join(pd.get_dummies(data[col], prefix=col))
        del data[col]
    
    data = data.dropna(axis=0)
        
    uncols = ['Age', 'Pclass', 'SibSp', 'Fare', 'Parch',]
    data[uncols] = (data[uncols] - data[uncols].mean() )/ data[uncols].std()

    return np.array(data.drop(['Survived', 'PassengerId'], axis=1), dtype=np.float16).T, np.array(data['Survived']).reshape(1, -1)

In [6]:
def train(X, y, val_X, val_y, hyp, forward, backward, accuracy):
    # W, b = np.random.rand(X.shape[0], y.shape[0]), 0
    W, b = np.zeros((X.shape[0], y.shape[0])), 0
    training_records = []

    for i in range(hyp['iters'] + 1):
        pred_y = forward(Weights=W, Bias=b, X=X)
        cost, W, b = backward(pred_Y=pred_y, grou_Y=y, Weights=W, Bias=b, X=X, hyp=hyp)

        if i%(hyp['iters']//10) == 0:
            val_pred_y = forward(Weights=W, Bias=b, X=val_X)
            val_cost = LogLoss(val_pred_y, val_y)
            training_records.append((i, cost, accuracy(pred_y, y), *confusion_matrix(pred_y, y)[1:], val_cost, accuracy(val_pred_y, val_y), *confusion_matrix(val_pred_y, val_y)[1:]))

    return W, b, training_records    

In [7]:
data = pd.read_csv('../train.csv')#.sample(frac=1)
train_X, train_Y = prepare_df(data[:700])
valid_X, valid_Y = prepare_df(data[700:])

In [8]:
forward = forward_prop
backward = momentum_gradient_descent
cost_function = LogLoss
accuracy = binary_accuracy

hyp = {
    'iters' : 500,
    'learning_rate' : 0.1,
    'm_train' : train_X.shape[1],
    'cost_function' : cost_function,
    'beta1' : 0.9,
    'beta2' : 0.9,
}

W, b, training_records = train(train_X, train_Y, valid_X, valid_Y, hyp, forward, backward, accuracy)
prepare_results(training_records).to_excel('./results.xlsx')

In [9]:
cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
pd.DataFrame(list(zip(cols, W.T[0])))

,0,1
0,Pclass,-0.928197
1,Age,-0.542612
2,SibSp,-0.300877
3,Parch,-0.107553
4,Fare,0.033504
5,Sex_female,1.231622
6,Sex_male,-1.275970
7,Embarked_C,0.156325
8,Embarked_Q,-0.050779
9,Embarked_S,-0.162739
